# Error Analysis


In [ ]:
# Ensure local package is importable
import sys, subprocess

print('Python:', sys.executable)
subprocess.run(['pip', 'install', '-e', '.'], check=True)


Quick error analysis on GBM predictions by hour and day-of-week.

In [ ]:
import pandas as pd
import pickle

# Load data splits
train = pd.read_parquet('data/processed/splits/train.parquet')
val = pd.read_parquet('data/processed/splits/val.parquet')
test = pd.read_parquet('data/processed/splits/test.parquet')

# Load GBM model bundle
bundle = pickle.load(open('artifacts/models/gbm_lightgbm_load_mw.pkl', 'rb'))
model = bundle['model']
feat_cols = bundle['feature_cols']

X = test[feat_cols].to_numpy()
y = test['load_mw'].to_numpy()
pred = model.predict(X)
resid = y - pred

err = test[['timestamp']].copy()
err['resid'] = resid
err['hour'] = pd.to_datetime(err['timestamp']).dt.hour
err['dow'] = pd.to_datetime(err['timestamp']).dt.dayofweek

err.groupby('hour')['resid'].mean().plot(title='Mean Residual by Hour')
